In [ ]:
%load_ext autoreload
%autoreload 2

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display

import sine_wave_speech.synthesis

In [ ]:
audio, _ = librosa.load("./data/sentence.wav", sr=8000)

In [ ]:
def play(audio, sr=8000):
    IPython.display.display(IPython.display.Audio(audio, rate=sr))

def specplot(audio, n_fft=512, sr=8000):
    S = librosa.stft(audio, n_fft=n_fft)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    librosa.display.specshow(S_db, y_axis="linear", sr=sr, n_fft=n_fft)

In [ ]:
specplot(audio)

In [ ]:
play(audio)

In [ ]:
from sine_wave_speech.lpc import fit_lpc, lpc_coefficients_to_frequencies

lpc_coefficients, gain, residual = fit_lpc(audio, p=8)

frequencies, magnitudes = lpc_coefficients_to_frequencies(lpc_coefficients, gain)
# Convert to Hz
frequencies = frequencies * 8000 / (2 * np.pi)

In [ ]:
plt.plot(frequencies[:100])
specplot(audio[128:100 * 128], n_fft=512)

In [ ]:
resynthesized = sine_wave_speech.synthesis.synthesize(
    frequencies, magnitudes, sr=8000, # wave_fn=lambda x: scipy.signal.sawtooth(x, width=.8)
)

play(resynthesized, sr=8000)